In [12]:
import pandas as pd
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score

df_submit = pd.read_csv('submission.csv')

f = open('final_data.pkl', 'rb')
f2 = open('final_test_data.pkl', 'rb')
data = pickle.load(f)
test_data = pickle.load(f2)

In [13]:
train_data, valid_data = train_test_split(data, test_size=0.01)
X_train, y_train = train_data.iloc[:, :100], train_data['bad_good']
X_valid, y_valid = valid_data.iloc[:, :100], valid_data['bad_good']
X_test = test_data

### 训练MLP&交叉验证后在测试集上预测

In [14]:
# 经测试，MLP网络结构如果较为简单或最大迭代步数较少
# 会由于目标列正负样本不平衡导致预测全0
model = MLPClassifier(hidden_layer_sizes=(10, 10, 5), solver='adam', max_iter=500)
model.fit(X_train, y_train)
score = cross_val_score(model, X_valid, y_valid, cv=5)
print(score.mean())     # average accuracy on valid dataset

y_pred = model.predict(X_test)
df_pred = pd.DataFrame({'bad_good': y_pred})
df_pred.insert(0, 'CUST_ID', df_submit['CUST_ID'])

0.9870304482748026


In [ ]:
df_pred['bad_good'].value_counts()

In [ ]:
df_pred.to_csv('result_MLP.csv', mode='w', index=False)
# 该文件提交后得分0.91967317497，说明预处理步骤具有一定可行性